In [1]:
from cameo import load_model
from cameo import fba

import escher
from escher import Builder

In [2]:
# load Chromobacterium model
model = load_model('../models/iDB858_updated1.xml')
model

Name,M_iDB858
Memory address,0x01f7062ceb20
Number of metabolites,1003
Number of reactions,1330
Number of groups,84
Objective expression,1.0*biomass1.12 - 1.0*biomass1.12_reverse_e00c6
Compartments,Cytoplasm


Number of metabolites: **1003**  
Number of reactions: **1330**

In [3]:
# simulasi model
s = fba(model)
flux = s.fluxes.to_frame()
s.objective_value

0.2515998481207547

In [4]:
# map from author
builder_scratch = Builder(
    map_json = '../maps/cv_Core metabolism_MAP.json',
    model = model,
    reaction_data = s.fluxes
)
builder_scratch

Builder(reaction_data={'rxn12008_c': 0.00233685938934557, 'rxn00541_c': 0.0, 'rxn00225_c': 0.0, 'rxn01674_c': …

Here we can see that many reactions and metabolite does not match map id

In [5]:
# Let's look at map data
import json
with open ("../maps/cv_Core metabolism_MAP.json", "r") as f:
    data = json.loads(f.read())

In [6]:
# how many reactions and metabolites in the map?
len_reaction = len(data[1]["reactions"])
len_metabolites = len([m for m in data[1]["nodes"] if data[1]["nodes"][m]["node_type"] == "metabolite"])
print("reactions:", len_reaction, "metabolites:", len_metabolites)

reactions: 139 metabolites: 357


## Mapping Metabolites
We would like to match the metabolites id with the metabolites in the model

In [7]:
# make a list of metabolites
met_model = {m.id: m.name for m in model.metabolites}
met_map = {data[1]["nodes"][m]['bigg_id']: data[1]["nodes"][m]["name"] for m in data[1]["nodes"] if data[1]["nodes"][m]["node_type"] == "metabolite"}

# find mismatch
mismatch_metabolites = [met for met in met_map.keys() if not met in met_model.keys()]
print(f"found {len(mismatch_metabolites)} mismatch:", mismatch_metabolites)

found 23 mismatch: ['34hpp_c', 'gln_DASH_L_c', 'glu_DASH_L_c', 'phe_DASH_L_c', 'tyr_DASH_L_c', 'phe_DASH_L_e', 'tyr_DASH_L_e', 'ser_DASH_L_c', 'trp_DASH_L_c', 'trp_DASH_L_e', '2i3ip_c', 'pdvnate_c', 'chpy_c', 'pvnate_c', 'vnate_c', 'vio_c', 'chpy_e', 'dvnate_c', 'dvio_c', 'dvio_e', 'provio_c', 'provio_e', 'vio_e']


Not bad, only 23 mismatches in the map, let's find the correct name in the model

In [8]:
for met in met_model.keys():
    for miss in mismatch_metabolites:
        query = miss.split("_")[0]
        if query in met_model[met]:
            print(query, met, met_model[met])

glu acgam1p_c N-Acetyl-D-glucosamine 1-phosphate
ser achms_c O Acetyl L homoserine C6H11NO4
glu dtdpglu_c DTDPglucose
glu sucglu_c N2-Succinyl-L-glutamate
glu sucgsa_c N2-Succinyl-L-glutamate 5-semialdehyde
glu uacgam_c UDP-N-acetyl-D-glucosamine
glu u3aga_c UDP-3-O-(3-hydroxytetradecanoyl)-N-acetylglucosamine
tyr ibcoa_c Isobutyryl-CoA
tyr ibcoa_c Isobutyryl-CoA
glu hpglu_c Tetrahydropteroyltri L glutamate C24H34N8O12
glu mhpglu_c 5 Methyltetrahydropteroyltri L glutamate C25H36N8O12
ser hom__L_c L-Homoserine
glu ugmd_c UDP-N-acetylmuramoyl-L-alanyl-D-gamma-glutamyl-meso-2,6-diaminopimelate
glu ugmda_c UDP-N-acetylmuramoyl-L-alanyl-D-glutamyl-meso-2,6-diaminopimeloyl-D-alanyl-D-alanine
glu akg_c 2-Oxoglutarate
glu uaagmda_c Undecaprenyl-diphospho-N-acetylmuramoyl-(N-acetylglucosamine)-L-ala-D-glu-meso-2,6-diaminopimeloyl-D-ala-D-ala
ser acser_c O-Acetyl-L-serine
tyr butACP_c Butyryl-ACP (n-C4:0ACP)
tyr butACP_c Butyryl-ACP (n-C4:0ACP)
glu g6p_B_c Beta D glucose 6 phosphate C6H11O9P
glu

Try to explore which one matches, sometimes need manual curation, and came up with this [table](../tables/metabolite_map.csv). Hint: the amino acid starts with uppercase. 

In [9]:
import pandas as pd

In [10]:
metabolite_map = pd.read_csv("../tables/metabolite_map.csv").dropna()
metabolite_map = metabolite_map.set_index("map_id")
metabolite_map

,map_name,model_id,model_name
map_id,,,
2i3ip_c,2i3ip[c],mDB_2i3ip_c,2 - imino - 3- (indol-3-yl)propanoate
34hpp_c,34hpp[c],2h34hppr_c,p-hydroxyphenylpyruvate
dvio_c,dvio[c],mDB_dvio_c,Deoxyviolacein
dvio_e,dvio[e],mDB_dvio_e,Deoxyviolacein
dvnate_c,dvnate[c],mDB_dvnate_c,Deoxyviolaceinate
gln_DASH_L_c,L-Glutamine,gln__L_c,L-Glutamine
glu_DASH_L_c,L-Glutamate,glu__L_c,L-Glutamate
pdvnate_c,pdvnate[c],mDB_pdvnate_c,Protodeoxyviolaceinate
phe_DASH_L_c,phe-L[c],phe__L_c,L-Phenylalanine


In [11]:
# replace metabolite value with model
for query in metabolite_map.index:
    for r in data[1]["reactions"]:
        for num, m in enumerate(data[1]["reactions"][r]["metabolites"]):
            if data[1]["reactions"][r]["metabolites"][num]["bigg_id"] == query:
                print("replacing:", query, "-->", metabolite_map.loc[query, "model_id"])
                #print(r, num, data[1]["reactions"][r]["metabolites"][num]["bigg_id"])
                data[1]["reactions"][r]["metabolites"][num]["bigg_id"] = metabolite_map.loc[query, "model_id"]
                print(r, num, "result:", data[1]["reactions"][r]["metabolites"][num]["bigg_id"])
                print("------------------")
    for n in data[1]["nodes"]:
        if data[1]["nodes"][n]["node_type"] == "metabolite":
            if data[1]["nodes"][n]["bigg_id"] == query:
                data[1]["nodes"][n]["bigg_id"] = metabolite_map.loc[query, "model_id"]
                data[1]["nodes"][n]["name"] = metabolite_map.loc[query, "model_name"]    
                print(n, data[1]["nodes"][n]["bigg_id"], metabolite_map.loc[query, "model_id"])

replacing: 2i3ip_c --> mDB_2i3ip_c
1577625 0 result: mDB_2i3ip_c
------------------
replacing: 2i3ip_c --> mDB_2i3ip_c
1577626 0 result: mDB_2i3ip_c
------------------
replacing: 2i3ip_c --> mDB_2i3ip_c
1577627 0 result: mDB_2i3ip_c
------------------
1577557 mDB_2i3ip_c mDB_2i3ip_c
replacing: 34hpp_c --> 2h34hppr_c
1577570 1 result: 2h34hppr_c
------------------
replacing: 34hpp_c --> 2h34hppr_c
1577575 1 result: 2h34hppr_c
------------------
1577233 2h34hppr_c 2h34hppr_c
replacing: dvio_c --> mDB_dvio_c
1577640 2 result: mDB_dvio_c
------------------
replacing: dvio_c --> mDB_dvio_c
1577641 1 result: mDB_dvio_c
------------------
1577660 mDB_dvio_c mDB_dvio_c
replacing: dvio_e --> mDB_dvio_e
1577641 0 result: mDB_dvio_e
------------------
replacing: dvio_e --> mDB_dvio_e
1577642 0 result: mDB_dvio_e
------------------
1577666 mDB_dvio_e mDB_dvio_e
replacing: dvnate_c --> mDB_dvnate_c
1577638 6 result: mDB_dvnate_c
------------------
replacing: dvnate_c --> mDB_dvnate_c
1577640 4 resu

In [12]:
with open('../results/edited_map.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [13]:
# map from author
builder_scratch = Builder(
    map_json = '../results/edited_map.json',
    model = model,
    reaction_data = s.fluxes
)
builder_scratch

Builder(reaction_data={'rxn12008_c': 0.00233685938934557, 'rxn00541_c': 0.0, 'rxn00225_c': 0.0, 'rxn01674_c': …

## Mapping Reactions

In [14]:
# What keys are available for each reactions?
data[1]["reactions"]["1576693"].keys()

dict_keys(['name', 'bigg_id', 'reversibility', 'label_x', 'label_y', 'gene_reaction_rule', 'genes', 'metabolites', 'segments'])

In [15]:
data[1]["reactions"]["1576693"]

{'name': 'Phosphoglycerate kinase',
 'bigg_id': 'PGK',
 'reversibility': True,
 'label_x': 1065,
 'label_y': 2715,
 'gene_reaction_rule': 'b2926',
 'genes': [{'bigg_id': 'b2926', 'name': 'pgk'}],
 'metabolites': [{'coefficient': -1, 'bigg_id': '3pg_c'},
  {'coefficient': 1, 'bigg_id': '13dpg_c'},
  {'coefficient': -1, 'bigg_id': 'atp_c'},
  {'coefficient': 1, 'bigg_id': 'adp_c'}],
 'segments': {'291': {'from_node_id': '1576835',
   'to_node_id': '1576836',
   'b1': None,
   'b2': None},
  '292': {'from_node_id': '1576836',
   'to_node_id': '1576834',
   'b1': None,
   'b2': None},
  '293': {'from_node_id': '1576485',
   'to_node_id': '1576835',
   'b1': {'y': 2822.4341649025255, 'x': 1055},
   'b2': {'y': 2789.230249470758, 'x': 1055}},
  '294': {'from_node_id': '1576486',
   'to_node_id': '1576835',
   'b1': {'y': 2825, 'x': 1055},
   'b2': {'y': 2790, 'x': 1055}},
  '295': {'from_node_id': '1576834',
   'to_node_id': '1576487',
   'b1': {'y': 2650, 'x': 1055},
   'b2': {'y': 2615, 'x

In [16]:
model.reactions.rxn12008_c

Reaction identifier,rxn12008_c
Name,rxn12008
Memory address,0x07f981e8450a0
Stoichiometry,hepdp_c + ipdp_c --> octdp_c + ppi_c All-trans-Heptaprenyl diphosphate + Isopentenyl diphosphate --> All-trans-Octaprenyl diphosphate + Diphosphate
GPR,CV_0847 or CV_2691 or CV_3954
Lower bound,0.0
Upper bound,1000.0


In [17]:
reaction_model = {}
for r in model.reactions:
    reaction_model[r.id] = {"name":r.name, "metabolites":[m.id for m in r.metabolites]}
#reaction_model

reaction_map = {}
for r in data[1]["reactions"]:
    reaction_map[data[1]["reactions"][r]["bigg_id"]] = {
        "name" : data[1]["reactions"][r]["name"],
        "metabolites": [i["bigg_id"] for i in data[1]["reactions"][r]["metabolites"]],
        "node": r}
#reaction_map

In [18]:
mismatch_reactions = []
for k in reaction_map.keys():
    if not k in reaction_model.keys():
        mismatch_reactions.append(k)

In [19]:
reaction_table = []
not_found = {}
for k in mismatch_reactions:
    print("searching for", k, "...")
    query = set(reaction_map[k]["metabolites"])
    for r in reaction_model:
        target = set(reaction_model[r]["metabolites"])
        if query == target:
            print(k, r)
            reaction_table.append({"map_id":k,
                                   "map_name": reaction_map[k]["name"],
                                   "map_metabolites": reaction_map[k]["metabolites"],
                                   "map_node": reaction_map[k]["node"],
                                   "model_id":r,
                                   "model_name": reaction_model[r]["name"],
                                   "model_metabolites": reaction_model[r]["metabolites"],
                                   })
        else:
            not_found[k] = "Not Found"
    print("----------")

searching for PGK ...
----------
searching for GND ...
----------
searching for O2t ...
O2t rxn05468_c
----------
searching for THD2 ...
THD2 rxn10125_c
----------
searching for FUM ...
FUM rxn00799_c
----------
searching for ICDHyr ...
----------
searching for PDH ...
PDH rDB00149_c
----------
searching for PPC ...
PPC rxn00251_c
----------
searching for ENO ...
ENO rxn00459_c
----------
searching for PGI ...
PGI rxn00558_c
----------
searching for NH4t ...
NH4t rxn05466_c
----------
searching for FBP ...
----------
searching for ICL ...
ICL rxn00336_c
----------
searching for ADK1 ...
ADK1 rxn00097_c
----------
searching for EX_gln__L_e ...
EX_gln__L_e EX_gln_L_e
----------
searching for TALA ...
TALA rxn01333_c
----------
searching for PYK ...
----------
searching for GLUt2r ...
GLUt2r rxn05297_c
----------
searching for EX_lac__D_e ...
EX_lac__D_e EX_lac_D_e
----------
searching for RPE ...
RPE rxn01116_c
----------
searching for SUCOAS ...
SUCOAS rxn00285_c
----------
searching fo

In [20]:
df_matching_reactions = pd.DataFrame(reaction_table)
# more than 1 match
drop = df_matching_reactions.map_id.value_counts()[df_matching_reactions.map_id.value_counts() > 1]
idx_filter = []
for idx in df_matching_reactions.index:
    if not df_matching_reactions.loc[idx, "map_id"] in drop:
        idx_filter.append(idx)
df_matching_reactions.loc[idx_filter, :]

,map_id,map_name,map_metabolites,map_node,model_id,model_name,model_metabolites
0,O2t,O2 transport diffusion,"[o2_c, o2_e]",1576695,rxn05468_c,O2 transport via diffusion,"[o2_e, o2_c]"
1,THD2,NAD P transhydrogenase,"[h_e, nadp_c, nadh_c, nadph_c, h_c, nad_c]",1576696,rxn10125_c,NADP transhydrogenase,"[nadh_c, nadp_c, h_e, h_c, nad_c, nadph_c]"
2,FUM,Fumarase,"[fum_c, h2o_c, mal__L_c]",1576697,rxn00799_c,S-Malate hydro-lyase,"[mal__L_c, fum_c, h2o_c]"
3,PDH,Pyruvate dehydrogenase,"[nadh_c, coa_c, pyr_c, accoa_c, nad_c, co2_c]",1576699,rDB00149_c,pyruvate dehydrogenase,"[coa_c, nad_c, pyr_c, co2_c, accoa_c, nadh_c]"
4,PPC,Phosphoenolpyruvate carboxylase,"[h2o_c, oaa_c, h_c, pep_c, pi_c, co2_c]",1576700,rxn00251_c,Orthophosphate oxaloacetate carboxyl-lyase pho...,"[co2_c, h2o_c, pep_c, h_c, pi_c, oaa_c]"
...,...,...,...,...,...,...,...
93,rxnvio8,rxnvio8,"[o2_c, h2o_c, mDB_dvio_c, h_c, mDB_dvnate_c, c...",1577640,rDB00097_c,Deoxyviolacein spontaneous synthesis,"[h_c, o2_c, mDB_dvnate_c, co2_c, h2o_c, mDB_dv..."
94,Ex_dvio_e,Ex_dvio[e],[mDB_dvio_e],1577642,EX_dvio_e,Deoxyviolacein Exchange,[mDB_dvio_e]
95,DF_provio,DF_provio,"[mDB_prodvio_c, mDB_prodvio_e]",1577644,rDB00100_c,Diffusion of prodeoxyviolacein,"[mDB_prodvio_c, mDB_prodvio_e]"
96,Ex_provio_e,Ex_provio[e],[mDB_prodvio_e],1577645,EX_prodvio_e,Prodeoxyviolacein Exchange,[mDB_prodvio_e]


In [21]:
#to do
## replace model reaction id with map
# load Chromobacterium model
model = load_model('../models/iDB858_updated1.xml')
for idx in df_matching_reactions.loc[idx_filter, :].index:
    target_model = df_matching_reactions.loc[idx, "model_id"]
    try:
        model.reactions.get_by_id(target_model).id = df_matching_reactions.loc[idx, "map_id"]
    except KeyError:
        print(target_model)   
    data[1]["reactions"][df_matching_reactions.loc[idx, "map_node"]]["name"] = df_matching_reactions.loc[idx, "model_name"]

with open('../maps/escher_map.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

rxn09272_c


In [22]:
# simulasi model
s = fba(model)
flux = s.fluxes.to_frame()
s.objective_value

0.2515998481207547

In [23]:
# map from author
builder_scratch = Builder(
    map_json = '../maps/escher_map.json',
    model = model,
    reaction_data = s.fluxes
)
builder_scratch

Builder(reaction_data={'rxn12008_c': 0.00233685938934557, 'rxn00541_c': 0.0, 'rxn00225_c': 0.0, 'rxn01674_c': …

## Final Cleaning

In [24]:
# double reaction with the same molecules, decide which one is which
drop

SUCCt2_2    2
SUCCt3      2
FUMt2_2     2
Name: map_id, dtype: int64

In [27]:
drop = df_matching_reactions.map_id.value_counts()[df_matching_reactions.map_id.value_counts() > 1]
idx_filter = []
for idx in df_matching_reactions.index:
    if df_matching_reactions.loc[idx, "map_id"] in drop:
        idx_filter.append(idx)
df_matching_reactions.loc[idx_filter, :].to_csv("../tables/double_reactions.csv")

In [28]:
df_matching_reactions.loc[idx_filter, :].map_id.unique()

array(['FUMt2_2', 'SUCCt3', 'SUCCt2_2'], dtype=object)

In [29]:
df_matching_reactions.loc[idx_filter, :].model_id.unique()

array(['rxn05561_c', 'rxn10152_c', 'rxn10154_c', 'rxn05654_c'],
      dtype=object)

In [30]:
reaction_model = {}
for r in model.reactions:
    reaction_model[r.id] = {"name":r.name, "metabolites":[m.id for m in r.metabolites]}
#reaction_model

reaction_map = {}
for r in data[1]["reactions"]:
    reaction_map[data[1]["reactions"][r]["bigg_id"]] = {
        "name" : data[1]["reactions"][r]["name"],
        "metabolites": [i["bigg_id"] for i in data[1]["reactions"][r]["metabolites"]],
        "node": r}
#reaction_map

mismatch_reactions = []
for k in reaction_map.keys():
    if not k in reaction_model.keys():
        mismatch_reactions.append(k)
print(len(mismatch_reactions))
mismatch_reactions

32


['PGK',
 'GND',
 'ICDHyr',
 'FBP',
 'PYK',
 'PFK',
 'PPCK',
 'PTAr',
 'ME1',
 'ACKr',
 'FUMt2_2',
 'SUCCt3',
 'GAPD',
 'SUCCt2_2',
 'GLUDy',
 'FORti',
 'FRUpts2',
 'SUCDi',
 'D_LACt2',
 'BIOMASS_Ecoli_core_w_GAM',
 'DDPA',
 'DHQS',
 'SHKK',
 'PSCVT',
 'CHORS',
 'PHEt2r',
 'rxnvio6',
 'DF_chpy',
 'Ex_chpy_e',
 'DF_dvio',
 'rxnvio9',
 'DF_vio']

In [32]:
df_missing_reactions = []
for i in data[1]["reactions"]:
    if data[1]["reactions"][i]["bigg_id"] in mismatch_reactions:
        df_missing_reactions.append(data[1]["reactions"][i])
df_missing = pd.DataFrame(df_missing_reactions)
df_missing = df_missing.loc[:, ["name", "bigg_id", "gene_reaction_rule", "genes", "metabolites"]]
df_missing.to_csv("../tables/missing_on_map.csv") 

In [33]:
model_reactions = []
for i in model.reactions:
    d = {"name":i.name, 
         "bigg_id":i.id,
         "metabolites":[{"coefficient":i.get_coefficient(x), "bigg_id":x.id} for x in i.metabolites]}
    model_reactions.append(d)
df_model_reactions = pd.DataFrame(model_reactions)
df_model_reactions.to_csv("../tables/all_reactions_in_model.csv")